<a href="https://colab.research.google.com/github/Atharva-Peshkar/DeepFake-Detector/blob/main/DeepFake_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import TimeDistributed
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Input
from keras.layers import Reshape
from keras.layers import GlobalAveragePooling2D

base_model = InceptionV3(input_shape = (299, 299, 3),include_top=False, weights='imagenet')

for layer_model in base_model.layers:
  layer_model.trainable = False


crnn = Sequential([
                   TimeDistributed(base_model,input_shape=(40,299,299,3)),   #Input to InceptionV3 wrapped in TimeDistributed Layer = (Timesteps,Height,Width,Chanels)
                   TimeDistributed(GlobalAveragePooling2D()),     #Output from InceptionV3 (Batch_Size,8,8,2048) passed through this to remove the spacial data and only keep the temporal data
                   LSTM(2048,dropout=0.5,input_shape=(40,2048)),  #Input Shape = (Timesteps,Features)
                   Dense(512),
                   Dropout(0.5),
                   Dense(2,activation="softmax")
])


print(crnn.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_16 (TimeDis (None, 40, 8, 8, 2048)    21802784  
_________________________________________________________________
time_distributed_17 (TimeDis (None, 40, 2048)          0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 2048)              33562624  
_________________________________________________________________
dense_16 (Dense)             (None, 512)               1049088   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 1026      
Total params: 56,415,522
Trainable params: 34,612,738
Non-trainable params: 21,802,784
_________________________________